In [186]:
import pandas as pd

In [187]:
with open("POS.train","r") as f:
    myfile = f.read()

In [188]:
lines = myfile.split("\n")

# Tasks

* Read, done
* Split to sentences Done
* Add a Label "Start/BEG" to denote beginning of the sentence
* While we are at it, add \</s> END to denote end of sentence

In [189]:
len(lines)

10001

In [190]:
def add_start_end(line):
    """
    Adds a START/START token at the beginning of the sentence and END/END token at the end of sentence
    """
    line = "linestartshere/START "+line+ "lineendshere/END"
    return line

In [191]:
lines= [add_start_end(l) for l in lines]

In [192]:
lines[0]

'linestartshere/START Pierre/NP Vinken/NP ,/, 61/CD years/NNS old/JJ ,/, will/MD join/VB the/DT board/NN as/IN a/DT nonexecutive/JJ director/NN Nov./NP 29/CD ./. lineendshere/END'

In [193]:
pos_unfiltered[0].split("/")

['Pierre', 'NP']

In [194]:
def split_label(tok):
    return tuple(tok.split("/"))

In [195]:
pos = [split_label(i) for line in lines for i in line.split(" ")]

In [196]:
pos[102][1]

'DT'

In [197]:
lis = []
for i in range(1,len(pos)):
    lis.append((pos[i-1][1],pos[i][1]))

In [198]:
lis[4]

('CD', 'NNS')

# Now we have tag bigrams

In [265]:
from collections import Counter
from collections import defaultdict

In [200]:
tag_emmission = Counter(lis)

In [201]:
tag_emmission[("NP","NP")]

9070

In [202]:
lpos = len(pos)

In [203]:
lpos

262263

In [204]:
def get_tag_transition_prob(tag1,tag2):
    return tag_emmission[(tag1,tag2)]/(lpos-1)

In [205]:
get_tag_transition_prob("START","NN")

0.0013688601474861016

# Now calculate tag emmission

* calculate total number of each tags.
* calculate word, tag count
* make function.

* possible tags for each word

In [206]:
tags = [i[1] for i in pos]

In [207]:
tag_count = Counter(tags)

In [208]:
tag_count["DT"]

21185

In [209]:
tag_emmission_counts = Counter(pos)

In [229]:
def get_tag_emmission_prob(word,tag):
    """NOTE D DEFINED BELOWWWWW"""
    return (tag_emmission_counts[(word,tag)]+1)/(tag_count[tag]+len(d))

In [211]:
get_tag_emmission_prob("THE","DT")

0.0003304224687278735

# What tags can a word take?

maintain a set for each word... we will need it.

ALSO for DEFAULT POLICY, ASSIGN NP if first letter capital otherwise NN.

In [212]:
d=dict()

In [213]:
for i in pos:
    if(i[0] in d):
        d[i[0]].add(i[1])
    else:
        d[i[0]] = set({i[1]})

In [214]:
d["START"]

{'NP'}

In [222]:
def default_policy(word):
    """
    General policy is that if the first char is uppercase make it a NP otherwise NN.
    We can change it in multiple ways but since most of the unknown might be NP or NN it would be fine.
    """
    if(word[0].isupper()):
        return {"NP"}
    else:
        return {"NN"}

In [223]:
def get_tag(word):
    if(word in d):
        return d[word]
    else:
        return default_policy(word)

# Actual Code for Viterbi

* tag_transition x tag_emmission

In [251]:
myword = "hope"
mytagset = {}


In [252]:
my_tags = get_tag(myword)

In [255]:
for tag in my_tags:
    print(tag,get_tag_emmission_prob(myword,tag))

VB 7.155379056205503e-05
VBP 0.0006561141638645124
NN 0.00023636363636363636


In [254]:
my_tags

{'NN', 'VB', 'VBP'}

In [257]:
lines[0]

'linestartshere/START Pierre/NP Vinken/NP ,/, 61/CD years/NNS old/JJ ,/, will/MD join/VB the/DT board/NN as/IN a/DT nonexecutive/JJ director/NN Nov./NP 29/CD ./. lineendshere/END'

In [258]:
my_test_str = "linestartshere Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov."

# My PLAN on how to implement it.

Ok. So we have this old_tag_set, we want to create a default dictionary that will keep maximum value for word_tag_set. so it will hold float, we want to keep max. transitions will be something like this, all elements in set a multiplied by all elements in set b.

Okay, so for the list part lets store the word 

Oh I so much wish dictionary were ordered so I could have passed two dictionary alongside...

DAMN too hard. I will have to make my own DataStructure I guess...

In [344]:
class POS_WORD():
    def __init__(self,tags,value,from_tag=""):
        self.current_value = value
        self.tags = tags
        self.from_tag = from_tag
        self.pval = 1
        
    def make_one_transition(self,tag1,tag2,word):
        """
        Simple, previously calculated max value * transition * emmission

        :params tag1,tag2,word,pval:
        :returns value:
        """
        return self.pval * get_tag_transition_prob(tag1,tag2) * get_tag_emmission_prob(word,tag2)
        
        
    def add_tag(self,tag,word):
        """Creates new entry"""
        self.pval = self.current_value
        self.from_tag = self.tags[-1] if len(self.tags) >= 1 else "START"
        self.current_value = self.make_one_transition(self.from_tag,tag,word)
        self.tags.append(tag)
    
    def update_tag(self,tag,word):
        """Updates existing entry if needed"""
        
        val = self.make_one_transition(self.from_tag,tag,word)
        if(self.current_value<val):
            self.current_value = val
            self.tags[-1] = tag
            
    def get_params(self):
        """
        I will use it to copy this class.
        """
        return self.tags.copy(), self.current_value
    
    
    

In [345]:
p = POS_WORD([],1,"")

In [346]:
p.add_tag("VB","hope")

In [347]:
p.get_params()

(['VB'], 9.276330078737563e-09)

In [348]:
p.from_tag

'START'

In [349]:
p.update_tag("NN","hope")

In [350]:
p.get_params()

(['NN'], 3.235487621330786e-07)

In [351]:
p.from_tag

'START'

In [352]:
# Second round

p.add_tag("DT","is")

In [353]:
p.get_params()

(['NN', 'DT'], 6.816745365399438e-15)

In [354]:
p.from_tag

'NN'

In [266]:
def returns_zero():
    return 0

In [259]:
def make_transition(old_tag_set={"START":POS_WORD([],1,"")}, word):
    # contains tag set for word under consideration
    word_tag_set = get_tag(word)
#     word_tag_value_holder = defaultdict(returns_zero)
    # lol idea of default dict looks neat but no please...
    word_tag_value_holder = dict()
    
    
    
    # should return current tag_set

In [263]:
def make_one_transition(tag1,tag2,word,pval=1):
    """
    Simple, previously calculated max value * transition * emmission
    
    :params tag1,tag2,word,pval:
    :returns value:
    """
    return pval * get_tag_transition_prob(tag1,tag2) * get_tag_emmission_prob(word,tag2)

In [264]:
make_one_transition("START","NN","Sarvesh") #Yay it works.

2.488836631792912e-08

# TRY 3

In [391]:
class POS_Holder():
    def __init__(self,tags,c_val):
        self.tags = tags
        self.c_val = c_val
        
    def get_last_tag(self):
        return self.tags[-1]
    
    def get_c_val(self):
        return self.c_val
    
    def get_params(self,tag,mval):
        t = self.tags.copy()
        t.append(tag)
        return t,mval
        

In [363]:
a= POS_Holder(["START"],1)

In [364]:
a.get_last_tag()

'START'

In [432]:
mval = 0

In [433]:
plis = [POS_Holder(["NN"],0.1),POS_Holder(["VB"],0.1)]

next_word = "hope"
# next_tag = "NN"

next_dict = dict()

word_tag_set = get_tag(next_word)
for next_tag in word_tag_set:
    for i in plis:

        calc = make_one_transition(i.get_last_tag(),next_tag,next_word,pval=i.get_c_val())

        if(calc>mval):
            mval=calc
            ref = i
    params = ref.get_params(next_tag,mval)
    next_dict[next_tag] = POS_Holder(params[0],params[1])
    


In [434]:
next_dict

{'VB': <__main__.POS_Holder at 0x7fed4e93ce10>,
 'VBP': <__main__.POS_Holder at 0x7fed483f3a10>,
 'NN': <__main__.POS_Holder at 0x7fed479af590>}

In [439]:
list(next_dict.values())[0].tags

['NN', 'VB']

In [417]:
# POS_Holder(params[0],params[1])
params

(['VB', 'NN'], 3.8573501446506305e-07)

In [380]:
mval

7.698477788692917e-07

In [442]:
def step(next_word,plis=[POS_Holder(["START"],1)]):
    next_dict = dict()
    word_tag_set = get_tag(next_word)
    for next_tag in word_tag_set:
        mval = 0
        for i in plis:
            calc = make_one_transition(i.get_last_tag(),next_tag,next_word,pval=i.get_c_val())
            if(calc>mval):
                mval=calc
                ref = i
        params = ref.get_params(next_tag,mval)
        next_dict[next_tag] = POS_Holder(params[0],params[1])
        
    return list(next_dict.values())

In [467]:
one = step("I")

In [468]:
one[0].tags

['START', 'NP']

In [469]:
one[1].tags

['START', 'PP']

In [471]:
two = step("am",one)

In [472]:
two[0].tags

['START', 'PP', 'VBP']

In [474]:
three = step("Sarvesh",two)

In [475]:
four = step("Bhatnagar",three)

In [477]:
four[0].c_val

2.3245807345404713e-25

In [478]:
four[0].tags

['START', 'PP', 'VBP', 'NP', 'NP']

In [485]:
def runner(my_test_str="Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov."):
    my_test_str = my_test_str.split(" ")
    plis = step(my_test_str[0])
    for i in range(1,len(my_test_str)):
        plis = step(my_test_str[i],plis)
    
    mmax = 0
    for i in plis:
        if(i.get_c_val() > mmax):
            ref = i
            mmax = i.get_c_val()
    return ref

In [491]:
l = runner("Hey there ! I am Sarvesh Bhatnagar and this is my POS tagger .")

In [492]:
l.tags

['START',
 'NP',
 'RB',
 'NN',
 'PP',
 'VBP',
 'NP',
 'NP',
 'CC',
 'DT',
 'VBZ',
 'PP$',
 'NP',
 'NN',
 '.']